<a href="https://colab.research.google.com/github/Vitor4442/TCC-VIS-O-COMPUTACIONAL/blob/master/MipNerf360.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/google-research/multinerf.git
%cd multinerf
!pip install -r requirements.txt

Cloning into 'multinerf'...
remote: Enumerating objects: 270, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 270 (delta 145), reused 117 (delta 117), pack-reused 88 (from 1)
Receiving objects: 100% (270/270), 174.34 KiB | 1.32 MiB/s, done.
Resolving deltas: 100% (157/157), done.
/content/multinerf
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.5 MB/s eta 0:00:00


###Alterações nas libs para se adequar ao python 3.12

In [2]:
%cd /content/multinerf
!sed -i "s/np\.math\.factorial/math.factorial/g" internal/ref_utils.py
!grep -q "^import math" internal/ref_utils.py || sed -i '1iimport math' internal/ref_utils.py


/content/multinerf


In [8]:
%cd /content/multinerf
!sed -i "s/INVALID_POINT3D = np\.uint64(-1)/INVALID_POINT3D = np.iinfo(np.uint64).max/g" internal/pycolmap/pycolmap/scene_manager.py


/content/multinerf


In [9]:
%cd /content/multinerf
!sed -i "s/^import math$/import math as pymath/g" internal/ref_utils.py
!sed -i "s/math\.factorial/pymath.factorial/g" internal/ref_utils.py


/content/multinerf


In [4]:
!git clone https://github.com/rmbrualla/pycolmap.git ./internal/pycolmap
!pip install -e internal/pycolmap

Cloning into './internal/pycolmap'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 68 (delta 11), reused 7 (delta 7), pack-reused 40 (from 1)
Receiving objects: 100% (68/68), 29.58 KiB | 14.79 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Obtaining file:///content/multinerf/internal/pycolmap
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for pycolmap (pyproject.toml) ... done
  Created wheel for pycolmap: filename=pycolmap-0.0.1-0.editable-py3-none-any.whl size=4540 sha256=64b2cae5b471a33f5b7e9b016095b544edfce71b776952b24114413b47b29067
  Stored in directory: /tmp/pip-ephem-wheel-cache-7phtwtmc/wheels/97/9b/11/900841e92a995eca0de49c0154b24c4217cd3f84569f3fcd33
Successfully built pycolmap


In [5]:
%cd /content/multinerf/

/content/multinerf


In [6]:
!touch tests/__init__.py

###Etapa de teste

In [10]:
!python -m unittest discover -s tests -p "*_test.py"


.............PE of degree 5 has a maximum error of 2.5369226932525635e-06
.PE of degree 10 has a maximum error of 6.4849853515625e-05
.PE of degree 15 has a maximum error of 0.002378210425376892
.PE of degree 20 has a maximum error of 0.11622805148363113
.PE of degree 25 has a maximum error of 1.999955415725708
.PE of degree 30 has a maximum error of 1.9999704360961914
....F................/content/multinerf/internal/math.py:28: RuntimeWarning: overflow encountered in cast
  return fn(jnp.where(jnp.abs(x) < t, x, x % t))
/usr/local/lib/python3.12/dist-packages/jax/_src/numpy/lax_numpy.py:2782: RuntimeWarning: overflow encountered in cast
  return util._where(condition, x, y)
............................................................................Mean Error = 0.08684495836496353, Tolerance = 0.1
.Mean Error = 0.08817846328020096, Tolerance = 0.1
...F.....................
FAIL: test_dataset_batch_creation (datasets_test.DatasetsTest.test_dataset_batch_creation)
datasets_test.Datasets

###Vamos iniciar o treinamento

###Instalando o dataset

In [12]:
!nvidia-smi

Thu Jan 15 23:36:11 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [22]:
!mkdir nerf_data
# A partir daqui eu fiz instalação do dataset no terminal mesmo
#wget http://storage.googleapis.com/gresearch/refraw360/360_v2.zip
#unzip 360_v2.zip

mkdir: cannot create directory ‘nerf_data’: File exists


In [14]:
%cd /content/multinerf

!python -m train \
  --gin_configs=configs/360.gin \
  --gin_bindings="Config.data_dir = 'data/garden'" \
  --gin_bindings="Config.dataset_loader = 'llff'" \
  --gin_bindings="Config.checkpoint_dir = 'checkpoints/garden_first'" \
  --gin_bindings="Config.max_steps = 200" \
  --logtostderr

/content/multinerf
2026-01-15 23:38:32.235842: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768520312.256685   16638 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768520312.262954   16638 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768520312.279525   16638 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768520312.279562   16638 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768520312.279566   16638 computation_placer.cc:177] com

In [25]:
!python -m train \
  --gin_configs=configs/360.gin \
  --gin_bindings="Config.data_dir = '/content/multinerf/nerf_data/stump'" \
  --gin_bindings="Config.dataset_loader = 'llff'" \
  --gin_bindings="Config.checkpoint_dir = '/content/multinerf/checkpoints/360/stump_smoke'" \
  --gin_bindings="Config.batch_size = 128" \
  --gin_bindings="Config.max_steps = 50" \
  --logtostderr


2026-01-16 00:15:55.070402: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768522555.090531   27113 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768522555.096572   27113 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768522555.112042   27113 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768522555.112069   27113 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768522555.112073   27113 computation_placer.cc:177] computation placer alr

In [26]:
!python -m render \
  --gin_configs=configs/360.gin \
  --gin_bindings="Config.data_dir = '/content/multinerf/nerf_data/stump'" \
  --gin_bindings="Config.dataset_loader = 'llff'" \
  --gin_bindings="Config.checkpoint_dir = '/content/multinerf/checkpoints/360/stump_smoke'" \
  --logtostderr


2026-01-16 00:23:04.218665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768522984.238777   29392 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768522984.244832   29392 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768522984.260246   29392 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768522984.260273   29392 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768522984.260277   29392 computation_placer.cc:177] computation placer alr

In [30]:
from IPython.display import Video, display

display(Video("/content/multinerf/checkpoints/360/stump_smoke/render/stump_smoke_360_test_preds_step_50_color.mp4", embed=True))
